In [1]:
import os, re
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm_notebook as tqdm
import matplotlib.pyplot as plt

In [2]:
os.chdir('..')

---

In [105]:
df = pd.read_csv(os.path.join(os.getcwd(), '3_pipeline', '1_intermediate', 'sample_raw.tsv'),
                 sep = '\t')

In [106]:
df = df.sort_values(by = ['gvkey', 'fyear'])

create loss sequence variable

In [107]:
# first create variable to check whether years are consecutive
df['L1_fyear'] = df.groupby('gvkey')['fyear'].shift(1)
df['is_year_cons'] = np.where((df['fyear'] - df['L1_fyear']) == 1, 1, 0)

In [108]:
out = []
loss_count = 0

for i, r in df.iterrows():
    if r['is_year_cons'] == 0:
        loss_count = 0
    elif pd.isnull(r['ib']):
        loss_count = 0
    elif r['ib'] >= 0:
        loss_count = 0
    elif r['ib'] < 0:
        loss_count += 1
    
    # only append from second loss onwards (otherwise 'sequential' doesn't really make sense?)
    # and: there is also a firstloss dummy in the data that captures the first instance of a loss
    loss_count_to_append = 0 if loss_count == 1 else loss_count
    
    out.append(loss_count_to_append)

In [109]:
df['lossseq'] = out

In [110]:
df['lossseq'] = np.where(df['ib'].isna(), np.nan, df['lossseq'])

create two-digit SIC

In [111]:
# drop if hsic is missing (very few obs)
df = df.loc[df['hsic'].notna()]

In [112]:
df['sic_group'] = df['hsic'].astype(int).astype(str).str.zfill(4).str[:2].astype(int)

In [113]:
df.to_stata(os.path.join(os.getcwd(), '3_pipeline', '1_intermediate', 'sample_dhaliwal.dta'),
            write_index = False)